In [ ]:
import os
import time
import numpy as np
import pandas as pd
import random
from tqdm import tqdm  # install
from multiprocessing import Pool
from torch.utils.data import Dataset
from PIL import Image
import torch
import multiprocessing as mp
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# Load image dataset: scene_graph

import json
import zipfile


def _load_scene_graph(scene_graph_path):
    """
    Load all JSON files in the scene_graph.zip file.
    Returns a dictionary where the keys are the JSON file names and the values are the parsed JSON objects.
    """    
    scene_graph = {}
    with zipfile.ZipFile(scene_graph_path) as zf:
        for filename in zf.namelist():
            if filename.endswith('.json'):
                with zf.open(filename) as f:
                    data = json.load(f)
                    image_id = data['image_id']
                    scene_graph[image_id] = data
    return scene_graph

In [ ]:
scene_graph_path="C:\\Users\\16377\\Downloads\\chest-imagenome-dataset-1.0.0\\chest-imagenome-dataset-1.0.0\\silver_dataset\\scene_graph.zip"
scene_graph = _load_scene_graph(scene_graph_path)

In [ ]:
# Convert image json file into base64str with dicom id

import base64
import json
import os
import zipfile
from PIL import Image
from io import BytesIO


def convert_images_to_base64(scene_graph_path, images_path):
    """
    Convert all images in the scene_graph.zip file to base64 strings.
    Returns a dictionary where the keys are the DICOM IDs and the values are the base64-encoded image strings.
    """
    base64_images = {}
        for filename in scene_graph:
            if filename.endswith('.json'):
                with zf.open(filename) as f:
                    data = json.load(f)
                    image_id = data['image_id']
                    subject_id = data['subject_id']
                    study_id = data['study_id']
                    dicom_id = image_id
                    file_path = os.path.join("files/p10/p", f"p{subject_id}", f"s{study_id}", f"{dicom_id}.jpg")
                    try:
                        with open(file_path, "rb") as image_file:
                            image_data = image_file.read()
                            base64_str = base64.b64encode(image_data).decode("utf-8")
                            base64_images[dicom_id] = base64_str
                    except FileNotFoundError:
                        pass
    return base64_images

#dicom id and filepath examples:
#subject_id	study_id	dicom_id	path	ViewPosition
#0	10000032	50414267	02aa804e-bde0afdd-112c0b34-7bc16630-4e384014	files/p10/p10000032/s50414267/02aa804e-bde0afd...	PA
#1	10000032	53189527	2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab	files/p10/p10000032/s53189527/2a2277a9-b0ded15...	PA
#2	10000032	53911762	68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714	files/p10/p10000032/s53911762/68b5c4b1-227d048...	AP
#3	10000032	53911762	fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818	files/p10/p10000032/s53911762/fffabebf-74fd3a1...	AP
#4	10000032	56699142	ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c	files/p10/p10000032/s56699142/ea030e7a-2e3b134...	AP

In [ ]:
# Create caption for each image

import pandas as pd

# Read the gold_sentence file
gold_sentence = pd.read_csv("C:\\Users\\16377\\Downloads\\chest-imagenome-dataset-1.0.0\\chest-imagenome-dataset-1.0.0\\gold_dataset\\gold_all_sentences_500pts_1000studies.txt", sep='\t', on_bad_lines='skip', header=0)

# Group the sentences by image_id and combine them into a single sentence
sentences = gold_sentence.groupby('image_id')['sentence'].apply(' '.join).reset_index()

# Save the sentences to a new CSV file with image_id and sentence columns
sentences.to_csv('sentences.csv', index=False)


In [ ]:
# Iterate all objects in each image, and get the box coordinators

def extract_all_objects(scene_graphs):
    """
    Extract all objects from all items in the input scene graphs and return them as a dictionary.
    The keys of the output dictionary are the names of the scene graph JSON files and the values
    are a list of dictionaries containing the object information.
    """
    all_objects = {}
    for image_id, data in scene_graphs.items():
        objects = []
        for item in data:
            for obj in item['objects']:
                obj_dict = {
                    'object_id': obj['object_id'],
                    'x1': obj['x1'],
                    'y1': obj['y1'],
                    'x2': obj['x2'],
                    'y2': obj['y2'],
                    'width': obj['width'],
                    'height': obj['height'],
                    'bbox_name': obj['bbox_name'],
                    'synsets': obj['synsets'],
                    'name': obj['name'],
                    'original_x1': obj['original_x1'],
                    'original_y1': obj['original_y1'],
                    'original_x2': obj['original_x2'],
                    'original_y2': obj['original_y2'],
                    'original_width': obj['original_width'],
                    'original_height': obj['original_height']
                }
                objects.append(obj_dict)
        all_objects[name] = objects
    return all_objects

# Object example in image json file:
#        {
#            "object_id": "d7bef063-28053f7a-f27dae40-4035348b-21a36d32_abdomen",
#            "x1": 39,
#            "y1": 118,
#            "x2": 179,
#            "y2": 223,
#            "width": 140,
#            "height": 105,
#            "bbox_name": "abdomen",
#            "synsets": [
#                "C0230168"
#            ],
#            "name": "Abdominal cavity",
#            "original_x1": 532,
#            "original_y1": 1350,
#            "original_x2": 2442,
#            "original_y2": 2783,
#            "original_width": 1910,
#            "original_height": 1433
#        },

In [ ]:
# Create caption for each object in each image
def extract_object_captions(scene_graph):
    """
    Creates captions for each object in each image json file
    Returns a dictionary where the keys are the image IDs and the values are dictionaries
    where the keys are the object IDs and the values are the corresponding captions.
    """
    object_captions = {}
    for image_id, image_data in scene_graph.items():
        attributes = image_data.get('attributes', [])
        for attribute in attributes:
            object_id = attribute.get('object_id')
            phrases = attribute.get('phrases', [])
            if object_id and phrases:
                # Combine the phrases into a single sentence
                caption = ' '.join(phrases)
                # Store the caption with the object ID as the key
                object_captions[f'obj_{object_id}'] = caption
    return object_captions